# PDF 파일을 사용한 RAG 구현

In [2]:
# !pip install pymupdf

In [3]:
from dotenv import load_dotenv
import os

import pymupdf as fitz

# .env 파일 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

#### PDF 파일 텍스트 추출 및 요약 

In [4]:
#  PDF 파일 텍스트 추출 함수
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text        

# OpenAI API 호출 텍스트 요약
def summarize_text(text):
    prompt = f"다음 문서를 요약해줘:\n\n{text}"
    
    response = client.chat.completions.create(
        model = "gpt-4o-mini", 
        messages = [ {"role":"user","content":prompt} ],
        temperature = 0.2
    )    

    return response.choices[0].message.content    

In [9]:
# PDF 파일 텍스트 추출
pdf_path = "기상조건에 따라 고도가 강수 특성에 미치는 영향_ 제주도 사례 분석.pdf"
full_text = extract_text_from_pdf(pdf_path)
print(full_text)

# 읽어온 PDF 텍스트를 텍스트 파일로 저장
txt_output_path = os.path.splitext(pdf_path)[0] + "_fulltext.txt"
with open(txt_output_path, "w", encoding="utf-8") as f:
    f.write(full_text)
print("텍스트 추출 및 저장 완료!!\n")

# 문서 요약
summary = summarize_text(full_text)
print("요약 결과:\n")
print(summary)

# 요약 결과를 텍스트 파일로 저장
summary_output_path = os.path.splitext(pdf_path)[0] + "_summary.txt"
with open(summary_output_path, "w", encoding="utf-8") as f:
    f.write(summary)  
print("텍스트 요약 및 저장 완료!!\n")

Atmosphere. Korean Meteorological Society
Vol. 35, No. 3 (2025) pp. 369-384
https://doi.org/10.14191/Atmos.2025.35.3.369
pISSN 1598-3560
eISSN 2288-3266
 2025 Korean Meteorological Society
369
기상조건에 따라 고도가 강수 특성에 미치는 영향: 제주도 사례 분석
이현정1),2) · 서명석1),2)*
1)국립공주대학교 대기과학과, 2)기상기후데이터 융합 분석 특성화 대학원
(접수일: 2025년 5월 21일, 수정일: 2025년 7월 3일, 게재확정일: 2025년 7월 12일)
The Influence of Altitude on Precipitation Characteristics Based
on Meteorological Conditions: A Case Study of Jeju Island
Hyeon-Jeong Lee1),2) and Myoung-Seok Suh1),2)*
1)Department of Atmospheric Science, Kongju National University, Gongju, Korea
2)Specialized Graduate School for Integrated Analysis of Meteorological and Climatic Data
(Manuscript received 21 May 2025; revised 3 July 2025; accepted 12 July 2025)
Abstract
In this study, the influence of altitude (Inf_o_Alt) on precipitation characteristics
(amount: Pr_Amt, frequency: Pr_Fre, intensity: Pr_Int) was investigated over various time
scales and meteorological conditions (wet/dry

### PDF 파일을 사용한 RAG 구현

In [10]:
from dotenv import load_dotenv
import os

import pymupdf as fitz

import faiss
import numpy as np
import pickle

# .env 파일 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

In [11]:
# 1) PDF 파일 텍스트 추출 함수
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text   

# 2) 텍스트를 청크로 나누는 함수
def split_text_into_chunks(text,chunk_size=1000,overlap=200):
    chunks = []
    start = 0

    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start = end - overlap   # 겹치는 부분을 두어 문맥 연결성 유지

    return chunks

In [13]:
# 3) OpenAI 임베딩 생성 함수 : 텍스트 리스트를 임베딩 벡터로 변환
def get_embeddings(texts):

    embeddings = []

    for text in texts:
        response = client.embeddings.create(
            model = 'text-embedding-3-small', 
            input = text        
        )
        embedding = response.data[0].embedding
        embeddings.append(embedding)
        
    return np.array(embeddings,dtype=np.float32)

# 4) FAISS 인덱스 생성 및 저장, 청크 저장 함수
def create_faiss_index_chunk(embeddings,chunks,index_path="faiss_index.bin",chunks_path="chunks.pkl"):

    # FAISS 인덱스 생성 (L2 거리 기반)
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)

    # 임베딩을 인덱스에 추가
    index.add(embeddings)

    # 인덱스와 청크 저장
    faiss.write_index(index,index_path)
    with open(chunks_path,'wb') as f:
        pickle.dump(chunks,f)

    print(f"FAISS 인덱스가 '{index_path}'에 저장되었습니다.")
    print(f"텍스트 청크가 '{chunks_path}'에 저장되었습니다.")
    
    return index

In [14]:
# 5) FAISS 인덱스와 청크 로드 함수
def load_faiss_index_chunk(index_path="faiss_index.bin",chunks_path="chunks.pkl"):
    index = faiss.read_index(index_path)
    with open(chunks_path,'rb') as f:
        chunks = pickle.load(f)

    return index,chunks

# 6) 질의에 대한 유사도 검색 함수  : 질의와 유사한 청크를 검색
def search_similar_chunks(query,index,chunks,top_k=3):

    # 질의를 임베딩으로 변환
    query_embedding = get_embeddings([query])

    # FAISS에서 유사한 벡터 검색
    distances, indices = index.search(query_embedding,top_k)

    # 검색된 청크들 반환
    similar_chunks = []
    for i, idx in enumerate(indices[0]):
        similar_chunks.append({
            'chunk': chunks[idx],
            'distance': distances[0][i]            
        })
    return similar_chunks

In [15]:
# 7) RAG 기반 답변 생성 함수
def generate_rag_answer(query,similar_chunks):

    # 컨텍스트 구성
    context = "\n\n".join([chunk['chunk'] for chunk in similar_chunks])

    # 프롬프트 구성
    prompt = f"""다음 문서 내용을 참고하여 질문에 답변해주세요.

    문서 내용:
    {context}
    
    질문: {query}
    
    답변:"""

    # OpenAI API 호출
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[ {"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=1000
    )
    
    return response.choices[0].message.content    

In [ ]:
# 8) 전체 RAG 시스템 실행 함수
def run_pdf_rag(pdf_path,query,rebuild_index=False):
    
    index_path = "faiss_index.bin"
    chunks_path = "chunks.pkl"

    # 인덱스가 없거나 재구축이 필요한 경우
    if rebuild_index or not os.path.exists(index_path):

        print("--> PDF에서 텍스트 추출 중...")
        full_text = extract_text_from_pdf(pdf_path)

        print("--> 텍스트를 청크로 분할 중...")
        chunks = split_text_into_chunks(full_text)

        print("--> 임베딩 생성 중...")
        embeddings = get_embeddings(chunks)

        print("--> FAISS 인덱스 생성 및 저장 중...")
        index = create_faiss_index_chunk(embeddings,chunks,index_path,chunks_path)
        

    

    
    